In [1]:
import cartopy.crs as ccrs
import intake
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import numpy as np
import xarray as xr
from tqdm.autonotebook import tqdm
import pyremo as pr
import cordex as cx
import glob
import regionmask
import regionmask
from REMO2020_mod import open_mfdataset

ERROR 1: PROJ: proj_create_from_database: Open of /work/ch0636/g300096/Python/envs/kernel2023/share/proj failed
/tmp/ipykernel_80432/1039813734.py:8: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
# create variable info class for model
class modelclass:
    def __init__(self, userexp, runame):
        self.userexp = userexp # XXXYYY
        self.runame = runame # name of the run

In [3]:
# define the runs to be plotted
modelruns =[] # init

modelruns.append(modelclass("056524","REMO2015"))
modelruns.append(modelclass("036030","REMO2020$_{27}$"))
modelruns.append(modelclass("036031","REMO2020$_{27}$ Shallow"))
modelruns.append(modelclass("036032","REMO2020$_{49}$"))
modelruns.append(modelclass("036033","REMO2020$_{49}$ no-Prog"))
modelruns.append(modelclass("036036","REMO2020$_{49}$ Wetcore"))
modelruns.append(modelclass("036037","REMO2020$_{49}$ Shallow"))

In [4]:
# datapath
dpath = "./data_in/model_data/histogram/"

In [5]:
# results path
respath = dpath

In [6]:
# RADKLIM mask path (if there is data (monthly sum) in a grid-box -> 1, if not 0)
rdklim_mask = dpath+"mm_masks/"

In [7]:
# define star year and end year
ystart=2001
yend=2010

In [8]:
# Set halo zone length (will be cutted out)
halo = 8

In [9]:
# we want to keep attributes when addid dataset
xr.set_options(keep_attrs=True)

In [10]:
# give parameters for the bins
bin_start = 0.0
bin_end = 140
bin_step = 1.0

In [11]:
# months for analysis
anmons = np.array([6,7,8])

In [12]:
for obj in modelruns: # here compute things are not really needed, probably
    # due to data size, we need to open year-by-year
    for year in range(ystart,yend+1):
        # load data
        ds =  pr.parse_dates(xr.open_dataset(dpath+"totpre2D_hourly_"+obj.userexp+"_"+str(year)+".nc",cache=False))
        # load mask data
        # filenames
        filenames = glob.glob(rdklim_mask+"EUR-11_"+str(year)+"*.nc")
        # load data
        dsm = open_mfdataset(filenames, parallel=False, chunks='auto')
        # combine data with masks and chooce only summer months
        dsmask = xr.merge([(ds["APRL"]+ds["APRC"]).rename("totpre").where(ds.time.dt.month.isin(anmons), drop=True), dsm.prec.where(dsm.time.dt.month.isin(anmons), drop=True).rename("rdklim_mask").sum(dim="time")], compat="override", join="override")
        # cut halo
        dsmask = dsmask.isel(rlat=slice(halo+1,dsmask.sizes["rlat"]-halo+1), rlon=slice(halo+1,dsmask.sizes["rlon"]-halo+1)).compute()
        # mask data
        final_data = dsmask.totpre.where(dsmask.rdklim_mask > 0, drop=True).rename("totpre").compute()
        # create histogram
        hist, bin_edges = np.histogram(final_data.values, bins=np.arange(bin_start,bin_end+bin_step,bin_step))
        xr_tmp = xr.Dataset(
            data_vars=dict(varri=(["bins"],hist,dict(standard_name="RADKLIM"))),
            coords=dict(bins=(["bins"],bin_edges[:-1]+bin_step/2.))
        ).compute()
        #
        if(ystart != yend): # more than one year
            if(year == ystart):
                collect_data = xr_tmp
            else:
                collect_data = collect_data + xr_tmp
        else: # only one year
            collect_data = xr_tmp
        # clear memory
        del ds, dsmask, dsm, final_data
    # write to file
    collect_data.to_netcdf(respath+"RADKLIM_1h_precipitation_hist_"+obj.userexp+"_"+str(ystart)+"-"+str(yend)+".nc",mode="w")
    # clear memory
    del collect_data